In [2]:
# !pip install -r requirements.txt

In [1]:
import pandas as pd
pd.read_parquet('../../session3/data/house_dataset_main.parquet').head(2)

,HouseId,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,EventTimestamp,Created
0,1,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.000,2021-12-11 18:40:03,2022-04-12 12:04:13
1,2,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,2.017,2021-12-11 18:57:30,2022-04-12 12:04:13


In [2]:
pd.read_parquet('../../session3/data/house_dataset_lat_lon.parquet').head(2)

,HouseId,Latitude,Longitude,EventTimestamp,Created
0,1,34.18,-118.38,2021-12-11 18:40:03,2022-02-13 22:43:53
1,2,39.08,-121.04,2021-12-11 18:57:30,2022-02-13 22:43:53


In [3]:
cd feature_store

/home/ubuntu/dvoitekh/aihouse_mlops_course/session4/feast/feature_store


In [4]:
!feast --help

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Usage: feast [OPTIONS] COMMAND [ARGS]...

  Feast CLI

  For more information, see our public docs at https://docs.feast.dev/

  For any questions, you can reach us at https://slack.feast.dev/

Options:
  -c, --chdir TEXT           Switch to a different feature repository
                             directory before executing the given subcommand.
  --log-level TEXT           The logging level. One of DEBUG, INFO, WARNING,
                             ERROR, and CRITICAL (case-insensitive).
  --feature-store-yaml TEXT  Override the directory where the CLI should look
                             for the feature_store.yaml file.
  --help                     Show this message and exit.

Commands:
  apply                    Create or 

In [8]:
!FEAST_S3_ENDPOINT_URL=http://localhost:9000 AWS_ACCESS_KEY_ID=minioadmin AWS_SECRET_ACCESS_KEY=minioadmin feast plan

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
10/27/2022 08:12:35 AM botocore.credentials INFO: Found credentials in environment variables.
Created entity HouseId
Created feature view house_lat_lon_view
Created feature view house_main_view
Created feature service house_service

No changes to infrastructure


In [9]:
!FEAST_S3_ENDPOINT_URL=http://localhost:9000 AWS_ACCESS_KEY_ID=minioadmin AWS_SECRET_ACCESS_KEY=minioadmin feast apply

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
10/27/2022 08:12:47 AM botocore.credentials INFO: Found credentials in environment variables.
Created entity HouseId
Created feature view house_main_view
Created feature view house_lat_lon_view
Created feature service house_service

Deploying infrastructure for house_main_view
Deploying infrastructure for hous

In [10]:
import os
from feast import FeatureStore

os.environ['FEAST_S3_ENDPOINT_URL'] = 'http://localhost:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minioadmin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minioadmin'

store = FeatureStore(repo_path=".")

/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


In [11]:
import feast
import pandas as pd
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

entity_df = pd.DataFrame.from_dict({"HouseId": [i for i in range(1, 1000)]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

retrieval_job = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
)

In [12]:
retrieval_job.metadata.features

['house_main_view:MedInc',
 'house_main_view:HouseAge',
 'house_main_view:AveRooms',
 'house_main_view:AveBedrms',
 'house_main_view:Population',
 'house_main_view:AveOccup',
 'house_main_view:MedHouseVal',
 'house_lat_lon_view:Latitude',
 'house_lat_lon_view:Longitude']

In [13]:
dataset = store.create_saved_dataset(
    from_=retrieval_job,
    name='merged_dataset_v1',
    storage=SavedDatasetFileStorage('s3://data/feast/merged_dataset_v1.parquet', s3_endpoint_override='http://localhost:9000')
)

training_df = retrieval_job.to_df()
print(len(training_df))
training_df.head()

/home/ubuntu/.local/lib/python3.9/site-packages/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


999


,HouseId,event_timestamp,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,Latitude,Longitude
0,1,2022-10-27 08:12:54.576596+00:00,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.000,34.18,-118.38
1,2,2022-10-27 08:12:54.576596+00:00,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,2.017,39.08,-121.04
2,3,2022-10-27 08:12:54.576596+00:00,3.7500,16.0,5.768719,1.023295,1478.0,2.459235,1.473,38.68,-121.28
3,4,2022-10-27 08:12:54.576596+00:00,2.8542,34.0,3.858779,1.045802,1164.0,4.442748,1.469,34.04,-118.19
4,5,2022-10-27 08:12:54.576596+00:00,1.3375,18.0,4.567625,1.087327,2707.0,2.882854,0.596,39.13,-121.54


after that, we can get the saved dataset via its name:

In [28]:
dataset = store.get_saved_dataset(name='merged_dataset_v1')
dataset.to_df()

/home/ubuntu/.local/lib/python3.9/site-packages/feast/feature_store.py:1235: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,MedInc,HouseAge,Population,MedHouseVal,HouseId,AveBedrms,Latitude,AveOccup,Longitude,event_timestamp,AveRooms
0,2.4792,24.0,2251.0,2.00000,1,1.134146,34.18,3.921603,-118.38,2022-10-27 08:12:54.576596+00:00,3.454704
1,5.8704,17.0,3174.0,2.34400,658,1.018360,37.60,3.884945,-122.06,2022-10-27 08:12:54.576596+00:00,6.314565
2,10.0595,26.0,573.0,5.00001,659,1.076923,34.19,3.148352,-118.90,2022-10-27 08:12:54.576596+00:00,8.692308
3,2.0133,35.0,2059.0,1.45800,660,0.957831,34.07,4.134538,-118.22,2022-10-27 08:12:54.576596+00:00,3.020080
4,5.4925,26.0,1195.0,3.61700,661,1.082228,33.82,3.169761,-118.31,2022-10-27 08:12:54.576596+00:00,6.220159
...,...,...,...,...,...,...,...,...,...,...,...
994,6.0389,5.0,591.0,2.15100,339,1.256410,37.70,3.788462,-121.44,2022-10-27 08:12:54.576596+00:00,8.750000
995,2.5272,5.0,2204.0,0.92000,340,1.402709,34.10,2.714286,-117.41,2022-10-27 08:12:54.576596+00:00,6.080049
996,6.2871,26.0,607.0,3.25500,341,1.073394,33.45,2.784404,-117.64,2022-10-27 08:12:54.576596+00:00,7.009174
997,2.9405,24.0,2937.0,2.17800,328,0.987437,34.08,3.689698,-118.14,2022-10-27 08:12:54.576596+00:00,3.767588


In [14]:
!feast materialize-incremental 2022-02-01T00:00:00

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
10/27/2022 08:12:56 AM botocore.credentials INFO: Found credentials in environment variables.
Materializing 2 feature views to 2022-02-01 00:00:00+00:00 into the redis online store.

Since the ttl is 0 for feature view house_main_view, the start date will be set to 1 year before the current time.
house_main_vi

In [15]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,MedInc,MedHouseVal,HouseAge,Longitude,Latitude
0,1,2.4792,2.0,24.0,-118.379997,34.18
1,20640,NaN,NaN,NaN,NaN,NaN


In [16]:
!feast materialize-incremental 2022-06-11T00:00:00

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
10/27/2022 08:13:10 AM botocore.credentials INFO: Found credentials in environment variables.
Materializing 2 feature views to 2022-06-11 00:00:00+00:00 into the redis online store.

house_main_view from 2022-02-01 00:00:00+00:00 to 2022-06-11 00:00:00+00:00:
10/27/2022 08:13:10 AM botocore.credentials INFO: F

In [17]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,MedInc,MedHouseVal,HouseAge,Longitude,Latitude
0,1,2.4792,2.000,24.0,-118.379997,34.180000
1,20640,3.3438,3.333,13.0,-117.800003,33.689999


let's use push source to immediately update both online and offline store

In [18]:
import datetime

store.push('main_push_source', pd.DataFrame([{
    'HouseId': 1,
    'MedInc': 1.0,
    'HouseAge': 1.0,
    'AveRooms': 1.0,
    'AveBedrms': 1.0,
    'Population': 1.0,
    'AveOccup': 1.0,
    'MedHouseVal': 1.0,
    'Latitude': 1.0,
    'Longitude': 1.0,
    'EventTimestamp': datetime.datetime.now(),
    'Created': datetime.datetime.now(),
}]))

In [19]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,MedInc,MedHouseVal,HouseAge,Longitude,Latitude
0,1,1.0000,1.000,1.0,-118.379997,34.180000
1,20640,3.3438,3.333,13.0,-117.800003,33.689999


unfortunately, it cannot update offline store in our case, cause it's a file. but it will update DB in case we use it for the offline store

In [20]:
entity_df = pd.DataFrame.from_dict({"HouseId": [1]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
).to_df()

,HouseId,event_timestamp,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,Latitude,Longitude
0,1,2022-10-27 08:13:22.060875+00:00,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.0,34.18,-118.38


In [23]:
!redis-cli INFO Keyspace

# Keyspace
db0:keys=20641,expires=0,avg_ttl=0


### Also, we can run Feast UI to track features and datasets

### And start the feature server via "feast serve -p 8001"  https://docs.feast.dev/reference/feature-servers/python-feature-server

In [33]:
import requests

requests.post(
    "http://localhost:8001/get-online-features",
    json={
        "feature_service": "house_service",
        "entities": {"HouseId": [1, 20640, 3434343]},
    }
).json()

{'metadata': {'feature_names': ['HouseId',
   'AveBedrms',
   'AveOccup',
   'Population',
   'AveRooms',
   'MedHouseVal',
   'MedInc',
   'HouseAge',
   'Latitude',
   'Longitude']},
 'results': [{'values': [1, 20640, 3434343],
   'statuses': ['PRESENT', 'PRESENT', 'PRESENT'],
   'event_timestamps': ['1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z']},
  {'values': [1.0, 0.9763513803482056, None],
   'statuses': ['PRESENT', 'PRESENT', 'PRESENT'],
   'event_timestamps': ['2022-10-27T08:13:22Z',
    '2022-06-09T18:13:41Z',
    '1970-01-01T00:00:00Z']},
  {'values': [1.0, 2.128378391265869, None],
   'statuses': ['PRESENT', 'PRESENT', 'PRESENT'],
   'event_timestamps': ['2022-10-27T08:13:22Z',
    '2022-06-09T18:13:41Z',
    '1970-01-01T00:00:00Z']},
  {'values': [1, 630, None],
   'statuses': ['PRESENT', 'PRESENT', 'PRESENT'],
   'event_timestamps': ['2022-10-27T08:13:22Z',
    '2022-06-09T18:13:41Z',
    '1970-01-01T00:00:00Z']},
  {'values': [1.0, 3.92229